In [5]:
import serial

In [6]:
# Custom Exceptions
class Error(Exception):
    """Base class for exceptions in this module.
    :param msg: Error message associated with the exception
    :type msg: str
    :ivar msg: Error message associated with the exception
    """

    def __init__(self, msg):
        self.msg = msg

    def __str__(self):
        return self.msg


class CommandError(Error):
    """Exception raised sending a command to the SAML11."""

    pass


class ConnectError(Error):
    """Exception raised connecting to the SAML11."""

    pass


# class TestAESError(Error):
#     """Exception raised when running the static AES test on a known block of data."""

#     pass

In [7]:
# https://github.com/nkinder/smart-card-removinator/blob/master/client/removinator/removinator.py
class SAML11:
    """Communicates to the SAML11 on the specified serial port.
    """

    def __init__(
        self,
        port="COM3",
        baudrate=9600,
        dsrdtr=True,
        bytesize=serial.EIGHTBITS,
        parity=serial.PARITY_EVEN,
        stopbits=serial.STOPBITS_TWO,
        DTR=True,
    ):
        """Opens a connection to the SAML11 on the specified serial port.
        """

        self.port = port
        self.baudrate = baudrate
        self.dsrdtr = dsrdtr
        self.bytesize = bytesize
        self.parity = parity
        self.stopbits = stopbits
        self.DTR = DTR

        self.last_result = ""
        self.last_response = ""

        # Open a connection to the SAML11.
        try:
            self.connection = serial.Serial(
                port=port,
                baudrate=baudrate,
                dsrdtr=dsrdtr,
                bytesize=bytesize,
                parity=parity,
                stopbits=stopbits,
            )
            self.connection.setDTR(DTR)
        except serial.SerialException as e:
            raise ConnectError(
                "Unable to open connection to SAML11 "
                "controller on port {0}".format(port)
            )

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.connection.close()

    def get_status(self):
        """
        Get the status of the SAML11
        :raises: :exc:`CommandError`
        """

        self.send_command("STA")
        # return json.loads(self.last_response.rstrip())
        return self.last_response

    def send_command(self, command, print_response=False):
        """
        Send a command to the SAML11
        Sends the specified command to the SAML11. The command
        result (*OK*, *DBG* or *ERR_\\**) will be available in the *last_result*
        attribute of your *SAML11* object.  Similarly, the full command
        response will be available in the *last_response* attribute.
        Any debug output will be printed.
        If an error result is encountered, a CommandError will be
        raised.
        :param command: the command to send
        :type command: str
        :raises: :exc:`CommandError`
        """

        # Clear out the previous command result and response.
        self.last_result = ""
        self.last_response = ""

        # Send the command in a proper envelope to the SAML11.
        try:
            self.connection.write("#{0}\r".format(command).encode("utf-8"))
        except serial.SerialException as e:
            raise CommandError(
                "Serial error encountered sending command "
                '"{0}" ({1})'.format(command, e)
            )

        while 1:
            resp_line = self.connection.readline().decode("utf-8")
            # print(resp_line)
            if resp_line.startswith("OK"):
                self.last_result = resp_line.rstrip()
                break
            elif resp_line.startswith("ERR_"):
                self.last_result = resp_line.rstrip()
                raise CommandError(
                    '{0} encountered sending command "{1}"'.format(
                        self.last_result, command
                    )
                )
            elif resp_line.startswith("[DBG]"):
                while 1:
                    print(resp_line.rstrip())
                    resp_line = self.connection.readline().decode("utf-8")
                    if resp_line.startswith("OK"):
                        self.last_result = "DBG"
                        break
            else:
                # Concatenate multi-line responses.
                self.last_response += resp_line

        if print_response:
            print("Last result: {}".format(self.last_result))
            print("Last response:\n{}".format(self.last_response).replace("\n", "\n\t"))

    def set_debug(self, debug):
        """
        Enable or disable SAML11 debug output
        When enabled, the SAML11 will return verbose responses
        via serial when commands are executed. The debug output from the
        previously run command will be available along with the standard
        command output in the *last_response* attribute of your
        *SAML11* object.
        :param debug: enable/disable debugging
        :type debug: bool
        :raises: :exc:`CommandError`
        """
        # Send the debug command.
        self.send_command("DBG")

        # Check the response to see if we are in the requested state.
        if debug and (self.last_response.rstrip().endswith("DBG_OFF")):
            self.send_command("DBG")
        elif not debug and (self.last_response.rstrip().endswith("DBG_ON")):
            self.send_command("DBG")

#     def test_aes(self):
#         """
#         Test AES encryption on a known block of data.
#         :raises: :exc:`TestAESError`
#         """
#         # Send the test AES command.
#         self.send_command("TAE")

In [9]:
with SAML11() as board:
    print(board.get_status())
    board.send_command("?", True)
    board.send_command("STA", True)
    board.set_debug(True)
    board.send_command("STA", True)
    board.set_debug(False)
    board.send_command("STA", True)
    board.send_command("TAE", True)
    board.send_command("TFL", True)
    board.send_command("511", True)

Status:	Debug mode is OFF

Last result: OK
Last response:
	--- SAML11 ---
	
	Command Format:
		Prefix:		#
		Terminator:	\r
	Commands:
		STA	Return debug status status
		DBG	Toggle debugging output
		TAE	Test AES on a known block of data
		TFL	Test Flash write and read using n bytes (command has to be followed by the number of bytes to be used in the test)
		?	Show usage information
	
Last result: OK
Last response:
	Status:	Debug mode is OFF
	
[DBG] Received command "STA"
Last result: OK
Last response:
	Status:	Debug mode is ON
	
[DBG] Received command "DBG"
Last result: OK
Last response:
	Status:	Debug mode is OFF
	
Last result: OK
Last response:
	key:		603deb1015ca71be2b73aef0857d77811f352c073b6108d72d9810a30914dff4
	iv:		000102030405060708090a0b0c0d0e0f
	input:		f58c4c04d6e5f1ba779eabfb5f7bfbd69cfc4e967edb808d679f777bc6702c7d39f23369a9d9bacfa530e26304231461b2eb05e2c39be9fcda6c19078c6a9d1b
	encrypted_ex:	ce054cfa6ec264de70202e906bd57065101da549b984ee26af6e06ef80fe30b3493431a21454391ac

SerialException: ClearCommError failed (PermissionError(13, 'The device does not recognize the command.', None, 22))